<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Alternate-Commits/blob/master/GC%20Fake%20real%20split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
!pip install -U sentence-transformers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import json

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output

In [3]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

# Load data

In [14]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/Thesis Consolidate/Datasets")
assert base_dir.exists()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
dataset_id = 'gossipcop'

In [16]:
import ast

converters = {"retweets":ast.literal_eval, "tweets":ast.literal_eval}
df = pd.read_csv(base_dir/f"{dataset_id}_agg_2.csv", converters=converters)
df.head()

,title,text,tweets,retweets,label,url
0,Kendall Kylie Jenner Jenner NOT Upset Up...,NaN,[],"[995423424741888001, 995461685166202880, 99987...",fake,NaN
1,Kim Kardashian Dethroned Dethroned By Khlo...,NaN,[],"[848843565027516416, 849030801970868224, 84884...",fake,NaN
2,Kim Kardashian Did NOT Hot Staffer Hot Sta...,NaN,[],"[940685393112064001, 977921622672920576, 94031...",fake,NaN
3,The Voice The Voice Team NOT Surprised Sur...,NaN,[],[],fake,NaN
4,Drake NOT Angelina Jolie s Toy Boy Toy Boy ...,NaN,"[{'id': 948630026496323585, 'text': 'Drake NOT...","[948022124626808832, 948630026496323585, 94801...",fake,NaN


In [17]:
len(df[df['label']=='real'])

15432

In [18]:
len(df[df['label']=='fake'])

4536